In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [1]:
import numpy as np
import pandas as pd
import time
from IPython.display import display 

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

In [2]:
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn import tree

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import RandomForestClassifier,  AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [4]:
dt = pd.read_pickle(r'data\data.pkl')

In [5]:
activities = dt.groupby('activity')

Just keep 4 activities



In [6]:
vals = ['standing', 'walking','running','cycling']
data = dt.loc[dt['activity'].isin(vals)].copy()

In [7]:
data.activity.value_counts()

walking     229709
standing    188984
cycling     163302
running      95641
Name: activity, dtype: int64

Cut off first and last 1000 items, because activity starts end ends



In [8]:
data.drop(data[data.activity=='running'].iloc[:1000].index, inplace=True)
data.drop(data[data.activity=='running'].iloc[-1000:].index, inplace=True)
data.drop(data[data.activity=='walking'].iloc[:1000].index, inplace=True)
data.drop(data[data.activity=='walking'].iloc[-1000:].index, inplace=True)
data.drop(data[data.activity=='cycling'].iloc[:1000].index, inplace=True)
data.drop(data[data.activity=='cycling'].iloc[-1000:].index, inplace=True)
data.drop(data[data.activity=='standing'].iloc[:1000].index, inplace=True)
data.drop(data[data.activity=='standing'].iloc[-1000:].index, inplace=True)

C:\Users\meesh\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [9]:
data.activity.value_counts()

walking     221175
standing    181109
cycling     151372
running      93113
Name: activity, dtype: int64

# Features

## Absolute Acceleration


absolute acceleration: $|a|=\sqrt{a_x^2 + a_y^2 + a_z^2}$

to get rid of the orientation of the device

In [10]:
def absacc(row):
    return np.sqrt(row['IMU_chest_ax1']**2 + row['IMU_chest_ay1']**2 + row['IMU_chest_az1']**2)/9.806

In [11]:
data['absacc'] = data.apply(absacc, axis=1)

C:\Users\meesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


## Max-Min Difference of absolute Acceleration

In [12]:
ws=512
dt = 1.0/100.0 # the activities were with 50Hz


In [13]:
data['accmax'] =  data['absacc'].rolling(window=ws,center=False).max() 
data['accmin'] = data['absacc'].rolling(window=ws,center=False).min() 

data['accmaxmindiff'] = data.accmax - data.accmin

C:\Users\meesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\meesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\meesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

## Fourier Transform of Rotation Rates


In [14]:
def fft_amplitude(s, kind='peak'):
    
    # don't forget the windowing to get rid of the leakage effect
    hann = np.hanning(len(s)) 
    
    # do the FFT with Hanning Window
    Yhann = np.fft.fft(hann*s)
    
    N = int(len(Yhann)/2+1)
    Y = 2*(np.abs(Yhann[:N])/N) # right half is enough info(positive freqs only)
    
    # frequency axis, if needed
    fa = 1.0/dt
    f = np.linspace(0, fa/2.0, N, endpoint=True)
    
    if kind=='peak':
        return np.max(Y) # just return the maximum peak amplitude
    elif kind=='periodicity':
        return np.max(Y) / np.mean(Y) # return periodicity
    elif kind=='full':
        return f, Y # return the full spectrum

In [15]:
data['fftamppeak'] = data['IMU_chest_rotz'].rolling(window=1*ws,center=False).apply(fft_amplitude, raw=False)


C:\Users\meesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Because of the rolling_ functions, there is overlap between the activity features and the labels, corresponding to it. We have to delete some rows (length of window), before using a classifier.



In [16]:
data.drop(data[data.activityID==3].iloc[0:int(ws)-1].index, inplace=True)
data.drop(data[data.activityID==4].iloc[0:int(ws)-1].index, inplace=True)
data.drop(data[data.activityID==5].iloc[0:int(ws)-1].index, inplace=True)
data.drop(data[data.activityID==6].iloc[0:int(ws)-1].index, inplace=True)

In [17]:
data.head()

,activityID,activity,heartrate,IMU_hand_temp,IMU_hand_ax1,IMU_hand_ay1,IMU_hand_az1,IMU_hand_ax2,IMU_hand_ay2,IMU_hand_az2,...,IMU_ankle_roty,IMU_ankle_rotz,IMU_ankle_magx,IMU_ankle_magy,IMU_ankle_magz,absacc,accmax,accmin,accmaxmindiff,fftamppeak
timestamp,,,,,,,,,,,,,,,,,,,,,
559.44,3,standing,101,32.6875,0.549558,7.35420,6.75881,0.484190,7.79716,6.62053,...,0.010350,-0.053009,-88.223999,32.996899,-4.05403,1.003447,1.048333,0.963532,0.084801,0.146737
559.45,3,standing,101,32.6875,0.513651,6.86262,6.91388,0.588009,7.32850,6.89326,...,0.040042,-0.028382,-87.089302,32.827202,-2.57978,0.999723,1.048333,0.963532,0.084801,0.146762
559.46,3,standing,101,32.6875,0.851353,6.70790,6.88097,0.707032,6.95026,7.09028,...,-0.024165,-0.043116,-87.096298,32.838799,-3.44605,1.005541,1.048333,0.963532,0.084801,0.146781
559.47,3,standing,101,32.6875,0.765876,6.81962,6.57141,0.750087,6.72339,7.06053,...,0.006977,-0.037264,-87.545700,32.640499,-3.93960,1.002116,1.048333,0.963532,0.084801,0.146793
559.48,3,standing,101,32.6875,0.798051,6.78005,6.41808,0.853185,6.63185,6.86432,...,0.012983,0.010073,-87.654099,32.099201,-4.43960,1.003846,1.048333,0.963532,0.084801,0.146798


In [18]:
data.to_pickle(r'data\data_feats.pkl')

# Classification

Each ensemble algorithm is demonstrated using 10 fold cross validation, a standard technique used to estimate the performance of any machine learning algorithm on unseen data.

In [19]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
num_trees = 100
max_features = 2

In [20]:
dict_classifiers = {
    "Logistic Regression": LogisticRegression(),
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors =3),
    #"Linear SVM": SVC(kernel='rbf', C=1.0, gamma=0.5),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Random Forest10": RandomForestClassifier(n_estimators=10),
    "Neural Net": MLPClassifier(alpha = 1),
    "Naive Bayes": GaussianNB(),
    
    #Bagging Algos
    "Bagged Decision Trees": BaggingClassifier(base_estimator=tree.DecisionTreeClassifier(), n_estimators=num_trees, random_state=seed),
    "RF": RandomForestClassifier(n_estimators=num_trees, max_features=max_features),
    "Extra Trees": ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features),
    #Boosting Algos
    "AdaBoost": AdaBoostClassifier(n_estimators=num_trees, random_state=seed),
    #Stochastic Gradient Boosting
    "Stochastic Gradient Boosting": GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
        
}

In [21]:
def batch_classify(X_train, Y_train, X_test, Y_test, no_classifiers = 5, verbose = True):
    """
    This method, takes as input the X, Y matrices of the Train and Test set.
    And fits them on all of the Classifiers specified in the dict_classifier.
    The trained models, and accuracies are saved in a dictionary. The reason to use a dictionary
    is because it is very easy to save the whole dictionary with the pickle module.
    
    Usually, the SVM, Random Forest and Gradient Boosting Classifier take quiet some time to train. 
    So it is best to train them on a smaller dataset first and 
    decide whether you want to comment them out or not based on the test accuracy score.
    """
    
    dict_models = {}
    for classifier_name, classifier in list(dict_classifiers.items())[:no_classifiers]:
        t_start = time.process_time()
        classifier.fit(X_train, Y_train)
        t_end = time.process_time()
        t_diff = t_end - t_start
        
        train_score = classifier.score(X_train, Y_train)
        test_score = classifier.score(X_test, Y_test)
        Y_true = classifier.predict(X_test)
        accuracy = accuracy_score(Y_true, Y_test)
        
        dict_models[classifier_name] = {'model': classifier, 'train_score': train_score, 'test_score': test_score, 'train_time': t_diff, 'y_true': Y_true, 'accuracy_score': accuracy, 'y_pred': Y_test}
        if verbose:
            print("trained {c} in {f:.2f} s".format(c=classifier_name, f=t_diff))
    return dict_models

In [22]:
def display_dict_models(dict_models, sort_by='test_score'):
    cls = [key for key in dict_models.keys()]
    test_s = [dict_models[key]['test_score'] for key in cls]
    training_s = [dict_models[key]['train_score'] for key in cls]
    training_t = [dict_models[key]['train_time'] for key in cls]
    accuracy_s = [dict_models[key]['accuracy_score'] for key in cls]
    
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),5)), columns = ['classifier', 'train_score', 'test_score', 'train_time','accuracy_score'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'train_score'] = training_s[ii]
        df_.loc[ii, 'test_score'] = test_s[ii]
        df_.loc[ii, 'train_time'] = training_t[ii]
        df_.loc[ii, 'accuracy_score'] = accuracy_s[ii]*100
    
    display(df_.sort_values(by=sort_by, ascending=False))

In [23]:
labels = data['activity'].values
np.shape(labels)

(638707,)

### Classification with Features

In [24]:
featurevector = ['accmaxmindiff','fftamppeak']

features = data[featurevector].values
np.shape(features)

(638707, 2)

In [25]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.4, random_state=42)

In [26]:
dict_models = batch_classify(features_train, labels_train, features_test, labels_test, no_classifiers = 11)

C:\Users\meesh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\meesh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


trained Logistic Regression in 3.91 s
trained Nearest Neighbors in 0.83 s
trained Decision Tree in 2.19 s
trained Random Forest10 in 11.55 s
trained Neural Net in 136.89 s
trained Naive Bayes in 0.89 s
trained Bagged Decision Trees in 142.30 s
trained RF in 141.56 s
trained Extra Trees in 51.17 s
trained AdaBoost in 95.30 s
trained Stochastic Gradient Boosting in 266.64 s


In [27]:
display_dict_models(dict_models)

,classifier,train_score,test_score,train_time,accuracy_score
8,Extra Trees,1.000000,0.997468,51.171875,99.746754
6,Bagged Decision Trees,0.999997,0.996329,142.296875,99.632852
7,RF,0.999995,0.996285,141.562500,99.628547
2,Decision Tree,1.000000,0.995597,2.187500,99.559658
3,Random Forest10,0.999559,0.991354,11.546875,99.135363
1,Nearest Neighbors,0.989356,0.980864,0.828125,98.086370
10,Stochastic Gradient Boosting,0.946126,0.944959,266.640625,94.495916
9,AdaBoost,0.921605,0.920652,95.296875,92.065225
4,Neural Net,0.917200,0.916061,136.890625,91.606095
5,Naive Bayes,0.878575,0.877851,0.890625,87.785097


In [28]:
import pickle
# Store data (serialize)
with open(r'data\2featclass.pickle', 'wb') as handle:
    pickle.dump(dict_models, handle)

# Load data (deserialize)
#with open('2featclass.pickle', 'rb') as handle:
 #   unserialized_data = pickle.load(handle)

### Classification without Features

In [29]:
x_cols = list(data.columns.values)
unwanted = {'activity','activityID','absacc','accmin','accmax','accmaxmindiff','fftamppeak'}
x_cols = [e for e in x_cols if e not in unwanted]
dt = data[x_cols].values
np.shape(dt)

(638707, 40)

In [30]:
df_train, df_test, y_train, y_test = train_test_split(dt, labels, test_size=0.4, random_state=42)

In [31]:
dict_models1 = batch_classify(df_train, y_train, df_test, y_test, no_classifiers = 11)
display_dict_models(dict_models1)

C:\Users\meesh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\meesh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


trained Logistic Regression in 649.08 s
trained Nearest Neighbors in 3.31 s
trained Decision Tree in 51.70 s
trained Random Forest10 in 62.73 s
trained Neural Net in 186.23 s
trained Naive Bayes in 1.22 s
trained Bagged Decision Trees in 3606.72 s
trained RF in 216.06 s
trained Extra Trees in 80.28 s
trained AdaBoost in 426.55 s
trained Stochastic Gradient Boosting in 1602.39 s


,classifier,train_score,test_score,train_time,accuracy_score
8,Extra Trees,1.000000,0.999996,80.281250,99.999609
1,Nearest Neighbors,0.999961,0.999937,3.312500,99.993737
10,Stochastic Gradient Boosting,0.999948,0.999898,1602.390625,99.989823
7,RF,1.000000,0.999875,216.062500,99.987475
3,Random Forest10,0.999997,0.999847,62.734375,99.984735
6,Bagged Decision Trees,1.000000,0.999656,3606.718750,99.965555
2,Decision Tree,1.000000,0.999241,51.703125,99.924065
4,Neural Net,0.996430,0.996270,186.234375,99.626981
0,Logistic Regression,0.981267,0.980457,649.078125,98.045663
5,Naive Bayes,0.955694,0.955954,1.218750,95.595402


In [32]:
# Store data (serialize)
with open(r'data\class.pickle', 'wb') as handle:
    pickle.dump(dict_models, handle)

# Load data (deserialize)
#with open('2featclass.pickle', 'rb') as handle:
 #   unserialized_data = pickle.load(handle)